In [10]:


import os
# Walk through  directory and list number of files
for dirpath, dirnames, filenames in os.walk("C:\\Users\\crack\\Downloads\\wheat_disease\\data\\train"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 3 directories and 0 images in 'C:\Users\crack\Downloads\wheat_disease\data\train'.
There are 0 directories and 902 images in 'C:\Users\crack\Downloads\wheat_disease\data\train\Brown_rust'.
There are 0 directories and 1116 images in 'C:\Users\crack\Downloads\wheat_disease\data\train\Healthy'.
There are 0 directories and 924 images in 'C:\Users\crack\Downloads\wheat_disease\data\train\Yellow_rust'.


In [12]:
import os
import numpy as np
import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from PIL import Image
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [13]:
data_dir='C:\\Users\\crack\\Downloads\\wheat_disease\\data\\train'
os.listdir(data_dir)

['Brown_rust', 'Healthy', 'Yellow_rust']

In [14]:
class_labels = ['Brown_rust', 'Healthy', 'Yellow_rust']

In [15]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Directories for dataset
TRAIN_DIR = 'C:\\Users\\crack\\Downloads\\wheat_disease\\data\\train'
VAL_DIR = 'C:\\Users\\crack\\Downloads\\wheat_disease\\data\\val'

# Image size and batch size
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32

# Define class labels
class_labels = ['Healthy', 'Brown_rust', 'Yellow_rust']

# Data augmentation and preprocessing for train and validation split
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation
)

datagen_val_test = ImageDataGenerator(rescale=1./255, validation_split=0.5)  # 50% validation, 50% test

# Creating train, validation, and test generators
train_generator = datagen_train.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = datagen_train.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

test_generator = datagen_val_test.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=42,
    subset='validation'
)

# Load EfficientNetB0 without the top layer
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Unfreeze last few layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze only the first few layers
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)  # Adding dropout to prevent overfitting
output = Dense(len(class_labels), activation='softmax')(x)  # 5 output classes

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=AdamW(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Compute class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Add EarlyStopping to avoid overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


Found 2355 images belonging to 3 classes.
Found 587 images belonging to 3 classes.
Found 368 images belonging to 3 classes.


In [16]:
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[early_stopping,lr_scheduler]
)

c:\wheat_disease\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.5531 - loss: 0.9398 - val_accuracy: 0.3799 - val_loss: 1.0496 - learning_rate: 1.0000e-04
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.6195 - loss: 0.8366 - val_accuracy: 0.5537 - val_loss: 0.9948 - learning_rate: 1.0000e-04
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.6057 - loss: 0.8458 - val_accuracy: 0.6167 - val_loss: 0.8928 - learning_rate: 1.0000e-04
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 282s 4s/step - accuracy: 0.6458 - loss: 0.7918 - val_accuracy: 0.6269 - val_loss: 0.8420 - learning_rate: 1.0000e-04
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 283s 4s/step - accuracy: 0.6536 - loss: 0.7794 - val_accuracy: 0.6831 - val_loss: 0.7503 - learning_rate: 1.0000e-04
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 298s 4s/step - accuracy: 0.6650 - loss: 0.7632 - val_accuracy: 0.4889 - val_loss: 0.8508 - learning_rate: 1.0000e-04
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 276s 4s/step - accuracy: 0.6600 

In [17]:

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.7038 - loss: 0.6493
Test Accuracy: 0.7147


In [20]:
# 13. Save the model
model.save("wheat_disease_fixed.h5")